In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [2]:
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)
word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx)
max_len = len(sentence.split())
n_hidden = 5


def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()
    for i in range(max_len - 1):
        input = [word2idx[n] for n in words[:(i + 1)]]
        input = input + [0] * (max_len - len(input))
        target = word2idx[words[i + 1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)


input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 16, True)

C:\Users\boss\AppData\Local\Temp\ipykernel_13672\388313711.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.Tensor(input_batch), torch.LongTensor(target_batch)


In [3]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # input : [max_len, batch_size, n_class]

        hidden_state = torch.randn(1 * 2, batch_size,
                                   n_hidden)  # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1 * 2, batch_size,
                                 n_hidden)  # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model


model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [4]:
for epoch in range(10000):
    for x, y in loader:
        pred = model(x)
        loss = criterion(pred, y)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 0.502151
Epoch: 1000 cost = 0.292523
Epoch: 2000 cost = 0.384019
Epoch: 2000 cost = 0.023474
Epoch: 3000 cost = 1.617565
Epoch: 3000 cost = 0.339231
Epoch: 4000 cost = 0.210180
Epoch: 4000 cost = 0.490827
Epoch: 5000 cost = 0.141607
Epoch: 5000 cost = 0.155279
Epoch: 6000 cost = 0.036562
Epoch: 6000 cost = 0.002418
Epoch: 7000 cost = 0.015197
Epoch: 7000 cost = 0.006828
Epoch: 8000 cost = 0.006215
Epoch: 8000 cost = 0.004603
Epoch: 9000 cost = 0.003079
Epoch: 9000 cost = 0.010708
Epoch: 10000 cost = 0.049682
Epoch: 10000 cost = 0.004516
GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more
['Actions', 'makes', 'it', 'easy', 'to', 'automate', 'all', 'your', 'software', 'workflows', 'from', 'continuous', 'integration', 'and', 'delivery', 'to', 'issue', 'triage', 'and', 'more']
